# Seattle AirBnB data

## Using data to understand the homeowner's market in Seattle

I approached the data as if I were a homeowner Seattle. If I were a homeowner in Seattle, my main objective would be to offer a great experience for my guests while making a healthy profit. Hence, I structured my business understanding questions around these objectives. My questions for my analysis are thus as follows: 

### Business Understanding:
1. Can we predict what drives higher ratings?
2. When are the most popular times of the year for Seattle home-owners?
3. When are the most profitable times of the year for Seattle home-owners?

### Data Understanding

#### Data Exploration

All data was obtained from Kaggle: https://www.kaggle.com/airbnb/seattle/home

In [351]:
#import libraries and load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
import collections
import sklearn
%matplotlib inline

listings = pd.read_csv('./seattle/listings.csv')
calendar = pd.read_csv('./seattle/calendar.csv')
reviews = pd.read_csv('./seattle/reviews.csv')

In [352]:
#explore columns in datasets
print(listings.columns.values)
print(calendar.columns.values)
print(reviews.columns.values)

['id' 'listing_url' 'scrape_id' 'last_scraped' 'name' 'summary' 'space'
 'description' 'experiences_offered' 'neighborhood_overview' 'notes'
 'transit' 'thumbnail_url' 'medium_url' 'picture_url' 'xl_picture_url'
 'host_id' 'host_url' 'host_name' 'host_since' 'host_location'
 'host_about' 'host_response_time' 'host_response_rate'
 'host_acceptance_rate' 'host_is_superhost' 'host_thumbnail_url'
 'host_picture_url' 'host_neighbourhood' 'host_listings_count'
 'host_total_listings_count' 'host_verifications' 'host_has_profile_pic'
 'host_identity_verified' 'street' 'neighbourhood'
 'neighbourhood_cleansed' 'neighbourhood_group_cleansed' 'city' 'state'
 'zipcode' 'market' 'smart_location' 'country_code' 'country' 'latitude'
 'longitude' 'is_location_exact' 'property_type' 'room_type'
 'accommodates' 'bathrooms' 'bedrooms' 'beds' 'bed_type' 'amenities'
 'square_feet' 'price' 'weekly_price' 'monthly_price' 'security_deposit'
 'cleaning_fee' 'guests_included' 'extra_people' 'minimum_nights'
 'm

It appears that all the datasets can potentially be merged by their listing ID, if needed during analysis. First, check that all columns are variables and rows are individuals.

In [353]:
#check no. of rows and columns
print(listings.shape)
print(calendar.shape)
print(reviews.shape)

(3818, 92)
(1393570, 4)
(84849, 6)


In [354]:
#check for missing values in the columns for each dataset, get percentages
(listings.isnull().sum()/len(listings)).sort_values(ascending=False)

license                             1.000000
square_feet                         0.974594
monthly_price                       0.602672
security_deposit                    0.511262
weekly_price                        0.473808
notes                               0.420639
neighborhood_overview               0.270299
cleaning_fee                        0.269775
transit                             0.244631
host_about                          0.224987
host_acceptance_rate                0.202462
review_scores_accuracy              0.172342
review_scores_checkin               0.172342
review_scores_value                 0.171818
review_scores_location              0.171556
review_scores_cleanliness           0.171032
review_scores_communication         0.170508
review_scores_rating                0.169460
reviews_per_month                   0.164222
first_review                        0.164222
last_review                         0.164222
space                               0.149031
host_respo

For the listing dataset, it looks as though there are a number of columns containing missing values. The license column is completely null.

In [355]:
(calendar.isnull().sum()/len(calendar)).sort_values(ascending=False)

price         0.32939
available     0.00000
date          0.00000
listing_id    0.00000
dtype: float64

For the calendar dataset, the price column has 32% of rows containing null values.

In [356]:
(reviews.isnull().sum()/len(reviews)).sort_values(ascending=False)

comments         0.000212
reviewer_name    0.000000
reviewer_id      0.000000
date             0.000000
id               0.000000
listing_id       0.000000
dtype: float64

The reviews dataset has almost no missing values.

In order to prepare the data for the 3 business questions, we need to look at the 3 datasets and determine which datasets and columns contained within them that are relevant to the question above.

We have 3 datasets: listings, calendar and reviews. Based on our brief exploration above, we can see that the dataset most relevant to our analysis for this question is the listings dataset. The calendar dataset looks to be more relevant to supplement the listings dataset for our 2nd question on popular times and availability. 

Meanwhile, the reviews dataset is more relevant for qualitative predictors and is mainly unstructured data, hence we will only analyse it if we lack sufficient information to answer our questions.

After determining the datasets that are relevant for answering our questions, we move to preparing the data for our analysis.

### Question 1: Can we predict what drives higher ratings?
#### Part I: Data Preparation

Seeing as there are many missing values in the license column, and it is not relevant to the questions above, we can drop it from our analysis dataset. 

In [357]:
#drop license column
listings.drop(columns=['license'],inplace=True)

Next, we revisit the question, which is on driving higher ratings in homes. The relevant column that can be seen as the target variable (y column) would be in the set of review_scores columns. However, we can see that there are several columns in the review_scores.

In [358]:
#check column names that begin with 'review_scores' 
[col for col in listings if col.startswith('review_scores_')]

['review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

Based on AirBnB's ratings methodology (https://www.airbnb.com/help/article/1257/how-do-star-ratings-work), the overall experience is the one that determines the overall experience for guests, and so the review_scores_rating column is the one I would set as my target variable.  

However, we need to revisit the other columns in the listings dataset. There are quite a few redundant columns that are unnecessary.

For example, it is unnecessary to have columns that only contain one unique value as they don't provide any predictive power.

In [359]:
#find columns in dataset that only contain one unique value
one_unique=[col for col in listings.columns.values if listings[col].nunique()==1]
one_unique

['scrape_id',
 'last_scraped',
 'experiences_offered',
 'market',
 'country_code',
 'country',
 'has_availability',
 'calendar_last_scraped',
 'requires_license',
 'jurisdiction_names']

Any columns that contain 'url' in the name are also irrelevant as they contain no predictive power or characteristics that lead to higher ratings for homes.

In [360]:
#find columns containing 'url' in the name
url_col=[col for col in listings.columns.values if 'url' in col]
url_col

['listing_url',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_url',
 'host_thumbnail_url',
 'host_picture_url']

In [361]:
#add url and single unique value to drop columns list
drop_cols=[]
drop_cols.extend(one_unique)
drop_cols.extend(url_col)
#drop columns from dataset
listings.drop(columns=drop_cols,inplace=True)

In [362]:
#get no. of columns
len(listings.columns.values)

73

For the remaining columns (ignoring the review_scores/target variable columns), we will need to look into a sample of one row from each column to determine if it would be valuable for our predictions or not.

In [363]:
listings_sample=[(x,listings[x][1]) for x in listings.columns.values if not x.startswith('review_scores_')]
listings_sample=pd.DataFrame(listings_sample, columns=['column_name','sample_value'])
listings_sample.set_index('column_name',inplace=True)
listings_sample

sample_value
column_name                                                                        
id                                                                           953595
name                                             Bright & Airy Queen Anne Apartment
summary                           Chemically sensitive? We've removed the irrita...
space                             Beautiful, hypoallergenic apartment in an extr...
description                       Chemically sensitive? We've removed the irrita...
neighborhood_overview             Queen Anne is a wonderful, truly functional vi...
notes                             What's up with the free pillows?  Our home was...
transit                           Convenient bus stops are just down the block, ...
host_id                                                                     5177328
host_name                                                                    Andrea
host_since                                                               2013-02-21
host_location                                    Seattle, Washington, United States
host_about                        Living east coast/left coast/overseas.  Time i...
host_response_time                                                   within an hour
host_response_rate                                                              98%
host_acceptance_rate                                                           100%
host_is_superhost                                                                 t
host_neighbourhood                                                       Queen Anne
host_listings_count                                                               6
host_total_listings_count                                                         6
host_verifications                ['email', 'phone', 'facebook', 'linkedin', 're...
host_has_profile_pic                                                              t
host_identity_verified                                                            t
street                            7th Avenue West, Seattle, WA 98119, United States
neighbourhood                                                            Queen Anne
neighbourhood_cleansed                                              West Queen Anne
neighbourhood_group_cleansed                                             Queen Anne
city                                                                        Seattle
state                                                                            WA
zipcode                                                                       98119
...                                                                             ...
accommodates                                                                      4
bathrooms                                                                         1
bedrooms                                                                          1
beds                                                                              1
bed_type                                                                   Real Bed
amenities                         {TV,Internet,"Wireless Internet",Kitchen,"Free...
square_feet                                                                     NaN
price                                                                       $150.00
weekly_price                                                              $1,000.00
monthly_price                                                             $3,000.00
security_deposit                                                            $100.00
cleaning_fee                                                                 $40.00
guests_included                                                                   1
extra_people                                                                  $0.00
minimum_nights                                                                    2
maximum_nights                         

From the sample data of one row for each column above, we can see some columns taht may not be relevant in the predicitive model. For example, the host particulars like host_id, host_name, host_since, host_location, host_about, host_verifications don't look like they will be useful, as they contain mainly unstructured and irrelevant information to our question of interest.

In [364]:
#remove relevant columns
drop_cols=['host_id', 'host_name', 'host_since', 'host_location', 'host_about', 'host_verifications']
listings.drop(columns=drop_cols,inplace=True)
listings_sample.drop(drop_cols,inplace=True)

Furthermore, there are some redundant columns that can be removed as well, for example the neighbourhood data (it looks like they have been aggregated into neighbourhood_cleansed which may be more useful).

In [367]:
#check neighbourhood columns to see which one would provide more information
print(listings['neighbourhood_cleansed'].unique())
print(listings['neighbourhood_group_cleansed'].unique())
print(listings['neighbourhood'].unique())

['West Queen Anne' 'Adams' 'West Woodland' 'East Queen Anne' 'Wallingford'
 'North Queen Anne' 'Green Lake' 'Westlake' 'Mann' 'Madrona'
 'University District' 'Harrison/Denny-Blaine' 'Minor' 'Leschi' 'Atlantic'
 'Pike-Market' 'Eastlake' 'South Lake Union' 'Lawton Park' 'Briarcliff'
 'Belltown' 'International District' 'Central Business District'
 'First Hill' 'Yesler Terrace' 'Pioneer Square' 'Gatewood' 'Arbor Heights'
 'Alki' 'North Admiral' 'Crown Hill' 'Fairmount Park' 'Genesee' 'Interbay'
 'Industrial District' 'Mid-Beacon Hill' 'South Beacon Hill' 'Greenwood'
 'Holly Park' 'Fauntleroy' 'North Beacon Hill' 'Mount Baker' 'Brighton'
 'South Delridge' 'View Ridge' 'Dunlap' 'Rainier Beach' 'Columbia City'
 'Seward Park' 'North Delridge' 'Maple Leaf' 'Ravenna' 'Riverview'
 'Portage Bay' 'Bryant' 'Montlake' 'Broadway' 'Loyal Heights'
 'Victory Heights' 'Matthews Beach' 'Whittier Heights' 'Meadowbrook'
 'Olympic Hills' 'Roosevelt' 'Lower Queen Anne' 'Wedgwood'
 'North Beach/Blue Ridge' 'C

After checking the neighbourhood columns above, I decided to use neighbourhood_cleansed instead of the other two variables as it provides a good balance of detail without being too noisy.

In [368]:
#remove redundant columns
drop_cols=['neighbourhood','host_neighbourhood','neighbourhood_group_cleansed']
listings.drop(columns=drop_cols,inplace=True)
listings_sample.drop(drop_cols,inplace=True)

Street, smart_location, latitude and longitude are all redundant data as we have information on location through other variables like neighbourhood, state and zip code. Thus they can be dropped. The variables first_review and last_review are dates that I am not going to focus on in my analysis for question 1.

In [373]:
#remove redundant data
drop_cols=['street','smart_location','latitude','longitude','first_review','last_review']
listings.drop(columns=drop_cols,inplace=True)
listings_sample.drop(drop_cols,inplace=True)

Next, to focus on all columns containing unstructured data (meaning containing sentences or text), which requires further feature engineering before they can be processed by the model. 

In [375]:
listings_sample

sample_value
column_name                                                                        
id                                                                           953595
name                                             Bright & Airy Queen Anne Apartment
summary                           Chemically sensitive? We've removed the irrita...
space                             Beautiful, hypoallergenic apartment in an extr...
description                       Chemically sensitive? We've removed the irrita...
neighborhood_overview             Queen Anne is a wonderful, truly functional vi...
notes                             What's up with the free pillows?  Our home was...
transit                           Convenient bus stops are just down the block, ...
host_response_time                                                   within an hour
host_response_rate                                                              98%
host_acceptance_rate                                                           100%
host_is_superhost                                                                 t
host_listings_count                                                               6
host_total_listings_count                                                         6
host_has_profile_pic                                                              t
host_identity_verified                                                            t
neighbourhood_cleansed                                              West Queen Anne
city                                                                        Seattle
state                                                                            WA
zipcode                                                                       98119
is_location_exact                                                                 t
property_type                                                             Apartment
room_type                                                           Entire home/apt
accommodates                                                                      4
bathrooms                                                                         1
bedrooms                                                                          1
beds                                                                              1
bed_type                                                                   Real Bed
amenities                         {TV,Internet,"Wireless Internet",Kitchen,"Free...
square_feet                                                                     NaN
price                                                                       $150.00
weekly_price                                                              $1,000.00
monthly_price                                                             $3,000.00
security_deposit                                                            $100.00
cleaning_fee                                                                 $40.00
guests_included                                                                   1
extra_people                                                                  $0.00
minimum_nights                                                                    2
maximum_nights                                                                   90
calendar_updated                                                              today
availability_30                                                                  13
availability_60                                                                  13
availability_90                                                                  16
availability_365                                                                291
number_of_reviews                                                                43
instant_bookable                                                                  f
cancellation_policy                    

Columns containing unstructured data like sentences will also not feature in the predictive model. Even though they may contain some important information, it would require to parse through those columns for commonly occurring words, so will require further feature engineering.

In [135]:
import string
listings_test=listings
listings_test['summary']=listings_test['summary'].dropna()
#remove punctuation
listings_test['description'] = listings_test['description'].str.replace('[^\w\s]','')

In [148]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
#add numbers to corpus
nums=[str(x) for x in range(1000)]
stop.extend(['seattle',"i've","i'm"])
stop.extend(nums)

In [150]:
lst1=collections.Counter(" ".join(listings_test["description"].dropna()).split()).most_common(1000)
[x for x in lst1 if x[0].lower() not in stop]

[('room', 3855),
 ('kitchen', 2715),
 ('home', 2660),
 ('bedroom', 2653),
 ('bed', 2516),
 ('downtown', 2500),
 ('apartment', 2350),
 ('house', 2170),
 ('space', 2069),
 ('access', 2014),
 ('neighborhood', 1944),
 ('private', 1927),
 ('bathroom', 1914),
 ('living', 1865),
 ('restaurants', 1843),
 ('one', 1824),
 ('bus', 1699),
 ('full', 1680),
 ('located', 1600),
 ('floor', 1549),
 ('walk', 1528),
 ('away', 1525),
 ('two', 1483),
 ('available', 1394),
 ('blocks', 1324),
 ('area', 1318),
 ('Hill', 1313),
 ('great', 1279),
 ('parking', 1245),
 ('queen', 1206),
 ('large', 1167),
 ('quiet', 1163),
 ('street', 1128),
 ('minutes', 1064),
 ('coffee', 1034),
 ('also', 1008),
 ('stay', 1005),
 ('Capitol', 1003),
 ('guests', 996),
 ('city', 969),
 ('comfortable', 965),
 ('distance', 955),
 ('walking', 947),
 ('new', 934),
 ('bath', 923),
 ('Lake', 914),
 ('unit', 865),
 ('close', 865),
 ('location', 859),
 ('TV', 851),
 ('use', 826),
 ('shops', 824),
 ('views', 803),
 ('light', 795),
 ('size', 7

In [76]:
listings_test['description']

0       Make your self at home in this charming one-be...
1       Chemically sensitive? We've removed the irrita...
2       New modern house built in 2013.  Spectacular s...
3       A charming apartment that sits atop Queen Anne...
4       Cozy family craftman house in beautiful neighb...
5       We're renting out a small private unit of one ...
6       Enjoy a quiet stay in our comfortable 1915 Cra...
7       Our tiny cabin is private , very quiet and com...
8       Nestled in the heart of the city, this space i...
9       Beautiful apartment in an extremely safe, quie...
10      Queen Anne Hill is a charming neighborhood wit...
11      Beautifully furnished, cozy 1 bedroom mid cent...
12      Spacious apt in popular Seattle neighborhood. ...
13      Enjoy our amazing, updated & modern design cot...
14      Stunning Designsponge featured 6 bed, 3.75 bat...
15      This home is full of light, art and comfort. 5...
16      Master bedroom suite with 1/4 bath & kitchenet...
17      Beauti

In [ ]:
bag_of_words = vec.transform(corpus)

In [ ]:
#make a copy to check for unstructured data


In [25]:
#drop unstructured data like 'summary', 'description', 'neighborhood_overview','transit',

TypeError: object of type 'float' has no len()

It also could be that columns like 'host_name' or 'host_about' are not particularly predictive of the rating as they are unstructured and not indicative of any characteristics of the home.

There are also redundant details like 'longitude' and 'latitude', and 

In [14]:
drop_cols=['listing_url','scrape_id','last_scraped','thumbnail_url',
       'medium_url', 'picture_url', 'xl_picture_url', 'host_id',
       'host_url', 'host_name','host_about','host_thumbnail_url',
       'host_picture_url','country_code','latitude',
       'longitude','latitude',
       'longitude','requires_license', 'jurisdiction_names']